In [ ]:
from IPython.display import Image
Image("/kaggle/input/xau-to-inr-image/Bitcoin_history_price.jpg")

# Overview

    Bitcoin is a decentralized digital currency that can be transferred on the peer-to-peer bitcoin network. Bitcoin transactions are verified by network nodes through cryptography and recorded in a public distributed ledger called a blockchain. The cryptocurrency was invented in 2008 by an unknown person or group of people using the name Satoshi Nakamoto.The currency began use in 2009, when its implementation was released as open-source software.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

file_path = 'btc-usd-max.csv'

# üìí  **<span style='color:#AE08F1'> 1. Load Bitcoin data </span>**

In [2]:
dataset = pd.read_csv(file_path)
dataset.head()

snapped_at   price    market_cap  total_volume
0  2013-04-28 00:00:00 UTC  135.30  1.500518e+09           0.0
1  2013-04-29 00:00:00 UTC  141.96  1.575032e+09           0.0
2  2013-04-30 00:00:00 UTC  135.30  1.501657e+09           0.0
3  2013-05-01 00:00:00 UTC  117.00  1.298952e+09           0.0
4  2013-05-02 00:00:00 UTC  103.43  1.148668e+09           0.0

In [3]:
dataset.describe()

price    market_cap  total_volume
count    4242.000000  4.241000e+03  4.242000e+03
mean    16883.203905  3.207177e+11  1.595336e+10
std     20907.654373  4.073053e+11  1.963780e+10
min        67.809000  7.713681e+08  0.000000e+00
25%       612.929000  8.638932e+09  3.355493e+08
50%      7694.431547  1.352809e+11  6.585688e+09
75%     27633.566396  5.359673e+11  2.615674e+10
max    101235.371703  2.003239e+12  1.904603e+11

# üìä  **<span style='color:#AE08F1'>2. EDA on Bitcoin Data </span>**

## üìó **<span style='color:#2980B9'>2.1 Check any null values present </span>**

In [4]:
dataset.isnull().sum()

snapped_at      0
price           0
market_cap      1
total_volume    0
dtype: int64

## üìó **<span style='color:#2980B9'>2.2 Plotting Few price data </span>**

In [5]:
import matplotlib.pyplot as plt
start_date = pd.to_datetime(dataset.date[0])
end_date = pd.to_datetime(dataset.date.values[-1])
dataset['date'] = pd.to_datetime(dataset['date'])

AttributeError: 'DataFrame' object has no attribute 'date'

In [ ]:
dataset.tail()

In [ ]:
top_plt = plt.subplot2grid((5,4), (0, 0), rowspan=3, colspan=4)
top_plt.plot(dataset.date, dataset["price"])
plt.title('Historical stock prices of Bitcoin [01-01-2015 to 20-10-2022]')
bottom_plt = plt.subplot2grid((5,4), (3,0), rowspan=1, colspan=4)
bottom_plt.bar(dataset.date, dataset['total_volume'])
plt.title('\nBitcoin Trading Volume', y=-0.60)
plt.gcf().set_size_inches(16,10)

In [ ]:
dataset.describe()

## üìó **<span style='color:#2980B9'>2.3 Check datatype of Adj Close price </span>**

In [ ]:
dataset['price'].dtype

## üìó **<span style='color:#2980B9'>2.4 Find Correlation  </span>**

In [ ]:
import seaborn as sns
# plotting correlation heatmap
plt.figure(figsize = (10, 6))
dataplot = sns.heatmap(dataset[['price', 'total_volume', 'market_cap']].corr(), cmap="BuPu", annot=True, 
                      fmt=".1f")
plt.show()

## üìó **<span style='color:#2980B9'>Obs -  </span>** 

    Here we can observe that, Price and Market_Cap are almost correlate features. 
    So for Price prediction, we can drop Market_Cap feature. 
    
**Selected Features**

* Price
* Total Volume

In [ ]:
dataset2 = dataset[['price', 'total_volume']]
dataset2.head()

## üìó **<span style='color:#2980B9'>2.5 Stationarity and detrending (ADF/KPSS)  </span>** 

Stationarity means that the statistical properties of a time series i.e. mean, variance and covariance do not change over time. Many statistical models require the series to be stationary to make effective and precise predictions.

Two statistical tests would be used to check the stationarity of a time series ‚Äì Augmented Dickey Fuller (‚ÄúADF‚Äù) test and Kwiatkowski-Phillips-Schmidt-Shin (‚ÄúKPSS‚Äù) test. A method to convert a non-stationary time series into stationary series shall also be used.

## üìó  **<span style='color:#2980B9'> 2.5.1 - ADF Test  </span>** 

ADF test is used to determine the presence of unit root in the series, and hence helps in understand if the series is stationary or not. The null and alternate hypothesis of this test are:

1. Null Hypothesis: The series has a unit root.
2. Alternate Hypothesis: The series has no unit root.

**üñäÔ∏è If the null hypothesis in failed to be rejected, this test may provide evidence that the series is non-stationary.**

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(dataset2.price.values, autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

## üìù **<span style='color:#2980B9'> 2.5.2 - Let's apply log transformation to the data and test again </span>** 

In [ ]:
from numpy import log

result = adfuller((log(dataset2.price.values)), autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

<div style="padding:20px;color:white;margin:0;font-size:250%;text-align:center;display:fill;border-radius:5px;background-color:#C29F10;overflow:hidden;font-weight:700;border: 5px solid #5c74f5;"> üìà Predict Bitcoin Future Price ü•á</div>

# üìô **<span style='color:#AE08F1'>3. Time Series Prediction </span>**

#### **<span style='color:#2980B9'> üñåÔ∏è We will try to predict the future prices of Bitcoin by using its Price feature. </span>**

  **<span style='color:#862DC3'>To perform forecasting, we will need a machine learning model. Most people think of multi-linear regression when they want to predict values. But for Time-series data, this is not a good idea. The main reason to not opt for regression for Time-Series Data is we are interested in predicting the future, which would be extrapolation (predicting outside the range of the data) for linear regression. And as we know that in linear regression any sort of extrapolation is not advisable. </span>**

 

# üìó **<span style='color:#2980B9'>What Model to Use?  </span>**

üìú  **<span style='color:#2980B9'>ARIMA  </span>**
    
üìú  **<span style='color:#2980B9'>NEURAL NETWORK  </span>**
    
üìú  **<span style='color:#2980B9'>RECURRENT NEURAL NETWORK  </span>**
    
üìú  **<span style='color:#2980B9'>LSTM - LONG SHORT TERM MEMORY  </span>**
    
üìú  **<span style='color:#2980B9'>CNN - CONVOLUTION NEURAL NETWORK  </span>**
    

# üìô **<span style='color:#AE08F1'>4. ARIMA - Univariate Price Forecasting  </span>**

    ARIMA is a class of models that ‚Äòexplains‚Äô a given time series based on its own past values, i.e, its own lags and the lagged forecast errors, so that equation can be used to forecast future values. Any ‚Äònon-seasonal‚Äô time series that exhibits patterns and is not a random white noise can be modeled with ARIMA models. The hypothesis testing (ADF) performed priorly, showed the prices were not stationary, hence we can not use an ARIMA model. We can use Neural networks for price predictions. 

#### **<span style='color:#2980B9'> Let's check how ARIMA works here  </span>** 

    To create an ARIMA model, we need 3 parameters:

* p; the order of the Auto-Regressive term
* q; the order of the Moving Average term
* d; the number of differencing required to make the time-series stationary

#### **<span style='color:#2980B9'> Determining Stationarity  </span>**  
    
    We will need the time-series data to be stationary because, the term Auto Regressive in ARIMA means it is a linear regression model that uses its own lags as predictors. Linear regression models, as you know, work best when the predictors are not correlated and are independent of each other.
    
    To achieve Stationarity, we need to perform differencing: subtract the previous value from the current value. Sometimes, depending on the complexity of the series, more than one differencing may be needed. Before we do this, we need to check if our data already is stationary or not. For this we will use the Augmented Dickey Fuller test. This is a Hypothesis test, assuming the Null hypothesis to be ‚ÄúA unit root is present in an Autoregressive model‚Äù. Unit Root tests whether a time series variable is non-stationary and possesses a unit root. If a series has a unit root, its shows a systematic pattern. The Alternate Hypothesis, in this case, will be that ‚ÄúNo unit root is present‚Äù, which means the time series is stationary.
    
* Data is not stationary, we can use differencing 2 times. 

####  **<span style='color:#2980B9'> Determining the order of the AR term  </span>**   

    p is the order of the Auto Regressive (AR) term. It refers to the number of lags of Y to be used as predictors. We can find out the required number of AR terms by inspecting the Partial Autocorrelation Function(PACF) plot. Partial autocorrelation can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags.
    
    


## **<span style='color:#2980B9'>4.1 - ARIMA model in words:  </span>** 

    Predicted Yt = Constant + Linear combination Lags of Y (upto p lags) + Linear Combination of Lagged forecast errors (upto q lags)

    The objective, therefore, is to identify the values of p, d and q. But how?

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})
# Original Series
fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(dataset2['price'].values); axes[0, 0].set_title('Original Series')
plot_acf(dataset2['price'].values, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(dataset2['price'].diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(dataset2['price'].diff().dropna(), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(dataset2['price'].diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(dataset2['price'].diff().diff().dropna(), ax=axes[2, 1])

plt.show()

In [ ]:
plt.rc("figure", figsize=(10,5))
plot_acf(dataset2['price'])
print()

In [ ]:
plt.rc("figure", figsize=(10,5))
plot_pacf(dataset2['price'])
print()

We can observe that the PACF lag 2 is the only one that is quite significant and it is well above the significance line compared to other values. Hence, we can safely set p to 2.

In [ ]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})

fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(dataset2['price'].diff()); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,5))
plot_pacf(dataset2['price'].diff().dropna(), ax=axes[1])

plt.show()

## üìó **<span style='color:#2980B9'> 4.2 - Determining the order of the MA term  </span>** 

    q is the order of the Moving Average (MA) term. It refers to the number of lagged forecast errors that should go into the ARIMA Model. An MA term is technically, the error of the lagged forecast.
    
    Just like how we looked at the PACF plot for the number of AR terms, you can look at the ACF plot for the number of MA terms. The ACF tells how many MA terms are required to remove any autocorrelation in the stationarized series.
    
    Count the number of lags that are well above the significance line. So, let‚Äôs tentatively fix q as 8.
    
   

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(dataset2['price'].diff()); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,1.2))
plot_acf(dataset2['price'].diff().dropna(), ax=axes[1])

plt.show()

In [ ]:
data = dataset2['price'].values
print('Length of Total data: ', len(data))
train_length = int(len(data) * 0.8)
train_data = data[:train_length]
test_data = data[train_length:]
print('Train and Test data length: ', len(train_data), len(test_data))

## üìó **<span style='color:#2980B9'> 4.3 - Building ARIMA Model  </span>**  

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(train_data, order=(1,0,8))
model_fit = model.fit(low_memory = False)
print(model_fit.summary())

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
# Forecast
forecast_result = model_fit.forecast(150, alpha=0.05)  # 95% conf
forecast_result[:30]

In [ ]:
test_data[:30]

## üìó **<span style='color:#2980B9'> 4.4 - Plot Test and Predicted Results  </span>**  

In [ ]:
# Here we're plotting Test and Predicted data

plt.figure(figsize=(12, 6))
plt.rcParams.update({'font.size': 12})
plt.plot(test_data[:150], '#0077be',label = 'Actual')
plt.plot(forecast_result[:], '#ff8841',label = 'Predicted')
plt.title('ARIMA Model for BITCOIN Price Forecasting')
plt.ylabel('BITCOIN Price [in Dollar]')
plt.xlabel('Time Steps [in Days] ')
plt.legend()
plt.show()

## üìó **<span style='color:#AE08F1'> Overview of ANN -  </span>**
Artificial Neural Networks (ANN) are multi-layer fully-connected neural nets that look like the figure below. They consist of an input layer, multiple hidden layers, and an output layer. Every node in one layer is connected to every other node in the next layer. We make the network deeper by increasing the number of hidden layers.

![](https://miro.medium.com/max/720/1*Gh5PS4R_A5drl5ebd_gNrg@2x.png)

    If we zoom in to one of the hidden or output nodes, what we will encounter is the figure below.
    
![](https://miro.medium.com/max/720/1*FcEfcrucAFymCr0gMFQ0QA@2x.png)   

A given node takes the weighted sum of its inputs, and passes it through a non-linear activation function. This is the output of the node, which then becomes the input of another node in the next layer. The signal flows from left to right, and the final output is calculated by performing this procedure for all the nodes. Training this deep neural network means learning the weights associated with all the edges.

The equation for a given node looks as follows. The weighted sum of its inputs passed through a non-linear activation function. It can be represented as a vector dot product, where n is the number of inputs for the node.

![](https://miro.medium.com/max/720/1*XBGeSr4IkHql0ZG_QnQu-Q@2x.png)

Backpropagation with gradient descent is literally the ‚Äúmagic‚Äù behind the deep learning models. It‚Äôs a rather long topic and involves some calculus, so we won‚Äôt go into the specifics in this applied deep learning series. 

**Common activation functions**

An activation function determines whether a neuron should be activated. The nonlinear functions typically convert the output of a given neuron to a value between 0 and 1 or -1 and 1. Some of the most commonly used functions are defined as follows:

**Sigmoid**: This is represented with the formula g(x) = 1/(1 + e^-x).

![](https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_02_13H-RecurrentNeuralNetworks-WHITEBG.png)

**Tanh**: This is represented with the formula g(x) = (e^-x - e^-x)/(e^-x + e^-x).

![](https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_02_13I-RecurrentNeuralNetworks-WHITEBG.png)

**Relu**: This is represented with the formula g(x) = max(0 , x)

![](https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_02_13J-RecurrentNeuralNetworks-WHITEBG.png)

# üìô **<span style='color:#AE08F1'> 5. Artificial Neural Network - Bitcoin price forecasting  </span>**

**Univariate Stock price forecasting**

    Using price data, we'll forecast the stock price for the next day. 

## üìó **<span style='color:#AE08F1'> 5.1 Load Data - "price" data  </span>**

In [ ]:
data = dataset2['price'].values
print('Shape of data: ', data.shape)

## üìó **<span style='color:#AE08F1'> 5.2 Separate Train and Test data  </span>**

In [ ]:
# Separate train and test data
train_length = int(len(data) * 0.8)
print('Train length: ', train_length)

train_data, test_data = data[:train_length], data[train_length:]
print('Shape of Train and Test data: ', train_data.shape, test_data.shape)

## üìó **<span style='color:#AE08F1'> 5.3 Change Shape - Need 2D data  </span>**

In [ ]:
train_data = train_data.reshape(-1, 1)
test_data = test_data.reshape(-1, 1)
print('Shape of Train and Test data: ', train_data.shape, test_data.shape)

## üìó **<span style='color:#AE08F1'> 5.4 Split a Univariate sequence to Supervised Learning  </span>**

In [ ]:
# split a univariate sequence into supervised learning [Input and Output]
def create_dataset(dataset, lookback):
    dataX, dataY = [], []
    for i in range(len(dataset) - lookback -1):
        a = dataset[i: (i+lookback), 0]
        dataX.append(a)
        b = dataset[i+lookback, 0]
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

## üìó **<span style='color:#AE08F1'> 5.5 Automatically select Lag value from PACF graph  </span>**

In [ ]:
plot_pacf(data, lags=10)
plt.show()

## üìó **<span style='color:#AE08F1'> 5.6 Taking Auto-correlation Lag value Greater than 10%  </span>**

In [ ]:
from statsmodels.tsa.stattools import pacf
pacf_value = pacf(data, nlags=20)
lag = 0
# collect lag values greater than 10% correlation 
for x in pacf_value:
    if x > 0.1:
        lag += 1
    else:
        break
print('Selected look_back (or lag = ): ', lag)

## üìó **<span style='color:#AE08F1'> 5.7 Separate Input and Output  </span>**

In [ ]:
train_X, train_y = create_dataset(train_data, lag)
test_X, test_y = create_dataset(test_data, lag)

In [ ]:
print('Shape of train_X and train_y: ', train_X.shape, train_y.shape)
print('Shape of test_X and test_y: ', test_X.shape, test_y.shape)

## üìó **<span style='color:#AE08F1'> 5.8 How Data Looks Like - Input and Output  </span>**

In [ ]:
print(train_data[:20])            # original data
for x in range(len(train_X[:20])):
    print(test_X[x], test_y[x], )            # trainX and trainY after lookback

## üìó **<span style='color:#AE08F1'> 5.9 Build an MLP model  </span>**

* **<span style='color:#862DC3'>Sequential()** </span> <span style='color:#108693'> - The sequential API allows you to create models layer-by-layer </span>

* **<span style='color:#862DC3'>add()** </span> <span style='color:#108693'> - To create a Sequential model incrementally via the add() method </span>

* **<span style='color:#862DC3'>Dense()** </span> <span style='color:#108693'> - To add a stack of fully connected layer (either a hidden layer or an output layer) </span>

* **<span style='color:#862DC3'>input_dim** </span>  <span style='color:#108693'> - To specify the input shape </span>

* **<span style='color:#862DC3'>Activation** </span> <span style='color:#108693'> - Decides the output a node. The purpose of the activation function is to introduce non-linearity into the output of a neuron. </span>

* **<span style='color:#862DC3'>Loss Function** </span> <span style='color:#108693'> - The function we want to minimize or maximize is called the objective function or criterion. When we are minimizing it, we may also call it the cost function, loss function, or error function. </span>

* **<span style='color:#862DC3'>Optimizer** </span> <span style='color:#108693'> - Minimizing the loss function, by changing the parameters (weights and biases) </span>

In [ ]:
# Fix random seed for reproducibility
# Thes seed value helps in initilizing random weights and biases to the neural network.  
np.random.seed(7)

In [ ]:
# ML libraries
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import keras

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim = lag, activation='relu', name= "1st_hidden"))
# model.add(Dense(64, activation='relu', name = '2nd_hidden'))
model.add(Dense(1, name = 'Output_layer', activation='linear'))
# model.add(Activation("linear", name = 'Linear_activation'))
model.compile(loss="mean_squared_error", optimizer="adam")
model.summary()

# üìó **<span style='color:#AE08F1'> 5.10 Fit data to Model  </span>**

In [ ]:
epoch_number = 100
batches = 64

history = model.fit(train_X, train_y, epochs = epoch_number, batch_size = batches, verbose = 1, shuffle=False, 
                    validation_split=0.1)

# üìó **<span style='color:#AE08F1'> 5.11 Train and Validation Loss  </span>**

In [ ]:
# plot history
plt.clf
plt.figure(figsize=(10,8))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.xlabel('Number of Epochs')
plt.ylabel('Train and Test Loss')
plt.title('Train and Test loss per epochs [Univariate]')
plt.legend()
plt.show()

## üìó **<span style='color:#AE08F1'> 5.12 Make Prediction  </span>**

In [ ]:
# Make prediction
testPredict = model.predict(test_X)
predicted_value = testPredict[:, 0]

## üìó **<span style='color:#AE08F1'> 5.13 Evaluation Metrics to Measure Performance  </span>**

* **<span style='color:#0A7681'> R-Squared </span>**

* **<span style='color:#0A7681'> Mean Absolute Error </span>**

* **<span style='color:#0A7681'> Mean Absolute Percentage Error</span>**

* **<span style='color:#0A7681'> Mean Squared Error</span>**

* **<span style='color:#0A7681'> Root Mean Squared Error</span>**

* **<span style='color:#0A7681'> Normalized Root Mean Squared Error</span>**

* **<span style='color:#0A7681'> Weighted Absolute Percentage Error</span>**

* **<span style='color:#0A7681'> Weighted Mean Absolute Percentage Error</span>**

Ref: https://analyticsindiamag.com/a-guide-to-different-evaluation-metrics-for-time-series-forecasting-models/

In [ ]:
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error

def evaluate_forecast_results(actual, predicted):
    print('R2 Score: ', round(r2_score(actual, predicted), 2))
    print('MAE : ', round(mae(actual, predicted), 2))
    print('MSE: ', round(mean_squared_error(actual,predicted), 2))
    print('RMSE: ', round(math.sqrt(mean_squared_error(actual,predicted)), 2))
    print('NRMSE: ', NRMSE(actual, predicted))
    print('WMAPE: ', WMAPE(actual, predicted))
    
def NRMSE(actual, predicted):
    rmse = math.sqrt(mean_squared_error(actual,predicted))
    nrmse = rmse / np.mean(actual)
    return round(nrmse, 4)

def WMAPE(actual, predicted):
    abs_error = np.sum(actual - predicted)
    wmape = abs_error / np.sum(actual)
    return round(wmape, 4)

In [ ]:
evaluate_forecast_results(test_y, predicted_value)

In [ ]:
# Here we're plotting Test and Predicted data

plt.figure(figsize=(16, 8))
plt.rcParams.update({'font.size': 12})
plt.plot(test_y[:], '#0077be',label = 'Actual')
plt.plot(predicted_value, '#ff8841',label = 'Predicted')
plt.title('MLP Model for BITCOIN Price Forecasting')
plt.ylabel('BITCOIN Price [in Dollar]')
plt.xlabel('Time Steps [in Days] ')
plt.legend()
plt.show()

A recurrent neural network (RNN) is a type of artificial neural network which uses sequential data or time series data. These deep learning algorithms are commonly used for ordinal or temporal problems, such as language translation, natural language processing (nlp), speech recognition, and image captioning; they are incorporated into popular applications such as Siri, voice search, and Google Translate.

Like feedforward and convolutional neural networks (CNNs), recurrent neural networks utilize training data to learn. They are distinguished by their ‚Äúmemory‚Äù as they take information from prior inputs to influence the current input and output. While traditional deep neural networks assume that inputs and outputs are independent of each other, the output of recurrent neural networks depend on the prior elements within the sequence. 

**Recurrent Neural Network vs. Feedforward Neural Network**

![](https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_02_13A-RecurrentNeuralNetworks-WHITEBG.png)

![](https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_02_13B-RecurrentNeuralNetworks-WHITEBG.png)

**Backpropagation through time (BPTT)**

Recurrent neural networks leverage backpropagation through time (BPTT) algorithm to determine the gradients, which is slightly different from traditional backpropagation as it is specific to sequence data. The principles of BPTT are the same as traditional backpropagation, where the model trains itself by calculating errors from its output layer to its input layer. 

These calculations allow us to adjust and fit the parameters of the model appropriately. BPTT differs from the traditional approach in that BPTT sums errors at each time step whereas feedforward networks do not need to sum errors as they do not share parameters across each layer.

**Vanishing gradients and Exploding gradients Problems**

RNNs tend to run into two problems, known as exploding gradients and vanishing gradients. These issues are defined by the size of the gradient, which is the slope of the loss function along the error curve. 

When the gradient is too small, it continues to become smaller, updating the weight parameters until they become insignificant‚Äîi.e. 0. When that occurs, the algorithm is no longer learning. 

Exploding gradients occur when the gradient is too large, creating an unstable model. In this case, the model weights will grow too large, and they will eventually be represented as NaN. One solution to these issues is to reduce the number of hidden layers within the neural network, eliminating some of the complexity in the RNN model.

![](https://stanford.edu/~shervine/teaching/cs-230/illustrations/architecture-rnn-ltr.png?9ea4417fc145b9346a3e288801dbdfdc)

# üìí 6. RNN - Univariate Time Series Forecasting

## üìó 6.1 Load Data - "Adj Close" price

In [ ]:
data = dataset2['price'].values
print('Shape of data: ', data.shape)

## üìó 6.2 Train - Test Split

In [ ]:
# Separate train and test data
train_length = int(len(data) * 0.8)
print('Train length: ', train_length)

train_data, test_data = data[:train_length], data[train_length:]
print('Shape of Train and Test data: ', len(train_data), len(test_data))

## üìó 6.3 Make time-series data supervised

In [ ]:
# split a univariate sequence into supervised learning [Input and Output]
from numpy import array
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

## üìó 6.4 Lag Value already Choosen from PACF Plot

In [ ]:
from statsmodels.tsa.stattools import pacf
pacf_value = pacf(data, nlags=20)
lag = 0
# collect lag values greater than 10% correlation 
for x in pacf_value:
    if x > 0.1:
        lag += 1
    else:
        break
print('Selected look_back (or lag = ): ', lag)

In [ ]:
n_features = 1

In [ ]:
train_X, train_y = split_sequence(train_data, lag)
test_X, test_y = split_sequence(test_data, lag)

In [ ]:
print('Shape of train_X and train_y: ', train_X.shape, train_y.shape)
print('Shape of test_X and test_y: ', test_X.shape, test_y.shape)

## üìó 6.5 Reshape train_X and test_X to 3-Dimension

In [ ]:
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], n_features))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], n_features))

In [ ]:
# New shape of train_X and test_X are :-
print('Shape of train_X and train_y: ', train_X.shape, train_y.shape)
print('Shape of test_X and test_y: ', test_X.shape, test_y.shape)

## üìó 6.6 Building and Defining the model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN
from keras.layers import Dense

In [ ]:
# define model
model = Sequential()
model.add(SimpleRNN(64, activation='relu', return_sequences=False, input_shape=(lag, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

## üìó 6.7 Fit the model - with training data

In [ ]:
# As you are trying to use function decorator in TF 2.0, 
# please enable run function eagerly by using below line after importing TensorFlow:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

## Callbacks API

A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc).

You can use callbacks to:

* Write TensorBoard logs after every batch of training to monitor your metrics
* Periodically save your model to disk
* Do early stopping
* Get a view on internal states and statistics of a model during training

Ref: https://keras.io/api/callbacks/#:~:text=A%20callback%20is%20an%20object,Do%20early%20stopping

In [ ]:
# fit model
cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
history = model.fit(train_X, train_y, epochs = 150, batch_size=64, verbose=1, validation_split= 0.1, 
                   callbacks=[cb])

## üìó 6.8 Summarize model accuracy and Loss

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## üìó 6.9 Make prediction - with Test data

In [ ]:
train_predict = model.predict(train_X)
test_predict = model.predict(test_X)

print('Shape of train and test predict: ', train_predict.shape, test_predict.shape)

## üìó 6.10 Model evaluation

In [ ]:
actual_ = test_y
predicted_ = test_predict[:, 0]
len(actual_), len(predicted_)

In [ ]:
evaluate_forecast_results(actual_, predicted_)

## 6.11 Plot test data and Predicted data


In [ ]:
plt.rc("figure", figsize=(14,8))
plt.rcParams.update({'font.size': 16})
plt.plot(actual_, label = 'Actual')
plt.plot(predicted_, label = 'Predicted')
plt.xlabel('Time in days')
plt.ylabel('BITCOIN price')
plt.title('BITCOIN price prediction using Simple RNN - Test data')
plt.legend()
plt.show()

In [ ]:
df_train = pd.DataFrame(columns = ['Train data'])
df_train['Train data'] = train_data

df = pd.DataFrame(columns = ['Test data', 'Predicted data'])
df['Test data'] = actual_
df['Predicted data'] = predicted_

total_len = len(df_train['Train data']) + len(df['Test data'])
range(len(df_train['Train data']), total_len)
x_list = [x for x in range(len(df_train['Train data']), total_len)]
df.index = x_list

In [ ]:
plt.rc("figure", figsize=(14,8))
plt.rcParams.update({'font.size': 16})
plt.xlabel('Time in days')
plt.ylabel('BITCOIN price')
plt.title('BITCOIN price prediction using Simple RNN')
plt.plot(df_train['Train data'])
plt.plot(df[['Test data', 'Predicted data']])
plt.legend(['Train', 'Test', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
data = dataset2['price'].values
print('Shape of data: ', data.shape)

In [ ]:
# Separate train and test data
train_length = int(len(data) * 0.8)
print('Train length: ', train_length)
train_data, test_data = data[:train_length], data[train_length:]
print('Shape of Train and Test data: ', len(train_data), len(test_data))

## üìó 7.3 Make univariate-series as Supervised Learning

In [ ]:
# split a univariate sequence into supervised learning [Input and Output]
from numpy import array
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

## üìó 4.4 Choose Lag value

In [ ]:
lag = 2  # Already this is calculated in 5.4 and 6.4 
n_features = 1

In [ ]:
train_X, train_y = split_sequence(train_data, lag)
test_X, test_y = split_sequence(test_data, lag)

print('Shape of train_X and train_y: ', train_X.shape, train_y.shape)
print('Shape of test_X and test_y: ', test_X.shape, test_y.shape)

## üìó 4.5 Reshape train_X and test_X to 3-Dimension

In [ ]:
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], n_features))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], n_features))

# New shape of train_X and test_X are :-
print('Shape of train_X and train_y: ', train_X.shape, train_y.shape)
print('Shape of test_X and test_y: ', test_X.shape, test_y.shape)

## üìó 5. Building a Deep Learning Model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

## üìó 5.1 Define the model

In [ ]:
# define model
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(lag, n_features)))
model.add(LSTM(64, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

## üìó 5.2 Fit model with data

In [ ]:
# As you are trying to use function decorator in TF 2.0, 
# please enable run function eagerly by using below line after importing TensorFlow:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
# fit model
cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
history = model.fit(train_X, train_y, epochs = 150, batch_size = 64, verbose=1, validation_split= 0.1, 
                   callbacks = [cb])

## üìó 5.3 Summarize model accuracy and Loss

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## üìó 5.4 Make Prediction

In [ ]:
train_predict = model.predict(train_X)
test_predict = model.predict(test_X)

print('Shape of train and test predict: ', train_predict.shape, test_predict.shape)

## üìó 5.5 Model Evaluation

In [ ]:
actual_lstm = test_y
predicted_lstm = test_predict[:, 0]
evaluate_forecast_results(actual_lstm, predicted_lstm)

In [ ]:
df_train = pd.DataFrame(columns = ['Train data'])
df_train['Train data'] = train_data

df = pd.DataFrame(columns = ['Test data', 'Predicted data'])
df['Test data'] = actual_lstm
df['Predicted data'] = predicted_lstm

total_len = len(df_train['Train data']) + len(df['Test data'])
range(len(df_train['Train data']), total_len)
x_list = [x for x in range(len(df_train['Train data']), total_len)]
df.index = x_list

In [ ]:
plt.rc("figure", figsize=(14,8))
plt.rcParams.update({'font.size': 16})
plt.xlabel('Time in days')
plt.ylabel('BITCOIN price')
plt.title('BITCOIN price prediction using LSTM')
plt.plot(df_train['Train data'])
plt.plot(df[['Test data', 'Predicted data']])
plt.legend(['Train', 'Test', 'Predictions'], loc='lower right')
plt.show()